# 環境設定

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [0]:
# 環境初始化 (大約三至五分鐘)
! wget -O init_env.sh https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh && \
bash init_env.sh

--2020-01-09 08:12:48--  https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6bnwn8u2hz19s59/init_env.sh [following]
--2020-01-09 08:12:48--  https://www.dropbox.com/s/raw/6bnwn8u2hz19s59/init_env.sh
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc70f9b05a586bf120546effe6bb.dl.dropboxusercontent.com/cd/0/inline/Av0AC-Hk3w4uCWAkqqwBSK8zDXEcH0bEwYMaRbVn5KOVztdSaVTaYAA_0GYITPbV7t_Vh6hhIgHhVL2S7tTx_IWZmYjDUcmE59J0zO6fmsssuA/file# [following]
--2020-01-09 08:12:48--  https://uc70f9b05a586bf120546effe6bb.dl.dropboxusercontent.com/cd/0/inline/Av0AC-Hk3w4uCWAkqqwBSK8zDXEcH0bEwYMaRbVn5KOVztdSaVTaYAA_0GYITPbV7t_Vh6hhIgHhVL2S7tTx_IWZmYjDUcmE59J0zO6fmsssuA/file
Resolving uc70f9b05a58

In [0]:
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.4-src.zip")

In [0]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [0]:
from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext()

# --------預處理部分--------


# 讀取資料

In [0]:
!wget -O pm25.csv "https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0"

--2020-01-09 08:13:02--  https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zkn3ba7pitv83el/pm2.5Taiwan.csv [following]
--2020-01-09 08:13:02--  https://www.dropbox.com/s/raw/zkn3ba7pitv83el/pm2.5Taiwan.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc26fa28cd9ec89a16c34e309504.dl.dropboxusercontent.com/cd/0/inline/Av0MYdRzOZ_7tqfdba4zh2u4emNf_lc30yujghKMfEozMHPZS7K2RyUiYXJ31zal8fHB9b-KwZhDZ0rhI196KFxMRo_xLYZHQJsfevku5TjTtQ/file# [following]
--2020-01-09 08:13:02--  https://uc26fa28cd9ec89a16c34e309504.dl.dropboxusercontent.com/cd/0/inline/Av0MYdRzOZ_7tqfdba4zh2u4emNf_lc30yujghKMfEozMHPZS7K2RyUiYXJ31zal8fHB9b-KwZhDZ0rhI196KFxMRo_xLYZHQJsfevku5TjTtQ/file
Resol

In [0]:
weather_data = sc.textFile("pm25.csv")

In [0]:
weather_data_rdd = weather_data.map(lambda line : line.split(","))

In [0]:
pm25schema = weather_data_rdd.first()
print (pm25schema)

['日期', '測站', '測項', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']


In [0]:
import math
def remove_row_with_noise (x):
    for i in range(3, len(x)):
        if not x[i].isdecimal():
            return False
    return True 

In [0]:
clean_weather_data = weather_data_rdd\
                    .filter(lambda x: x!=pm25schema)\
                    .filter(remove_row_with_noise)

In [0]:
#print(set(clean_weather_data.map(lambda x : x[1]).collect()))

In [0]:
list1 = list(set(clean_weather_data.map(lambda x : x[1]).collect()))

In [0]:
#str1 =''
#for i in range(75):
#  str2 = "x[1] == '" + list1[i] + "' or "
#  str1 += str2
#print(str1[:-3])

In [0]:
#len(set(clean_weather_data.map(lambda x : x[1]).collect()))

In [0]:
dalipm25 = clean_weather_data

In [0]:
#dalipm25 = clean_weather_data.filter(lambda x :x[1] == '士林' or x[1] == '埔里' or x[1] == '小港' or x[1] == '觀音' or x[1] == '花蓮' or x[1] == '松山' or x[1] == '新竹' or x[1] == '陽明' or x[1] == '新莊' or x[1] == '安南' or x[1] == '忠明' or x[1] == '屏東' or x[1] == '平鎮' or x[1] == '頭份' or x[1] == '金門' or x[1] == '前金' or x[1] == '中山' or x[1] == '善化' or x[1] == '崙背' or x[1] == '淡水' or x[1] == '林園' or x[1] == '冬山' or x[1] == '馬公' or x[1] == '鳳山' or x[1] == '沙鹿' or x[1] == '潮州' or x[1] == '菜寮' or x[1] == '新營' or x[1] == '新店' or x[1] == '仁武' or x[1] == '臺西' or x[1] == '彰化' or x[1] == '三義' or x[1] == '汐止' or x[1] == '苗栗' or x[1] == '關山' or x[1] == '大里' or x[1] == '湖口' or x[1] == '南投' or x[1] == '馬祖' or x[1] == '宜蘭' or x[1] == '板橋' or x[1] == '美濃' or x[1] == '橋頭' or x[1] == '斗六' or x[1] == '復興' or x[1] == '大寮' or x[1] == '楠梓' or x[1] == '龍潭' or x[1] == '大同' or x[1] == '豐原' or x[1] == '朴子' or x[1] == '嘉義' or x[1] == '麥寮' or x[1] == '桃園' or x[1] == '新港' or x[1] == '左營' or x[1] == '基隆' or x[1] == '土城' or x[1] == '古亭' or x[1] == '中壢' or x[1] == '永和' or x[1] == '萬里' or x[1] == '西屯' or x[1] == '前鎮' or x[1] == '二林' or x[1] == '林口' or x[1] == '臺南' or x[1] == '線西' or x[1] == '臺東' or x[1] == '大園' or x[1] == '三重' or x[1] == '竹東' or x[1] == '恆春' or x[1] == '萬華' )

In [0]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
dalipm25row = dalipm25.map(lambda p:
        Row(
        date = p[0],
        location = p[1],
        measure = p[2],
        hr_01 = float(p[3]),hr_02 = float(p[4]),hr_03 = float(p[5]),hr_04 = float(p[6]),hr_05 = float(p[7]),
        hr_06 = float(p[8]),hr_07 = float(p[9]),hr_08 = float(p[10]),hr_09 = float(p[11]),hr_10 = float(p[12]),
        hr_11 = float(p[13]),hr_12 = float(p[14]),hr_13 = float(p[15]),hr_14 = float(p[16]),hr_15 = float(p[17]),
        hr_16 = float(p[18]),hr_17 = float(p[19]),hr_18 = float(p[20]),hr_19 = float(p[21]),hr_20 = float(p[22]),
        hr_21 = float(p[23]),hr_22 = float(p[24]),hr_23 = float(p[25]),hr_24 = float(p[26]),
    )
)


#dalipm25row.take(10)

In [0]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

In [0]:
df = spark.createDataFrame(dalipm25row)

In [0]:
#df.show()

In [0]:
schemea = ["date", "location", "measure", 
           "hr_01", "hr_02", "hr_03", "hr_04",
           "hr_05", "hr_06", "hr_07", "hr_08",
           "hr_09", "hr_10", "hr_11", "hr_12",
           "hr_13", "hr_14", "hr_15", "hr_16",
           "hr_17", "hr_18", "hr_19", "hr_20",
           "hr_21", "hr_22", "hr_23", "hr_24",           
          ]

In [0]:
df = dalipm25.toDF(schemea)

In [0]:
#df.show()

In [0]:
for i in df.columns[3:]:
  df = df.withColumn(i, df[i].cast("float"))

In [0]:
#df.show()

In [0]:
'''
df_pm10_12 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    #df.hr_01.alias("hr_01_pm10"),\
                    #df.hr_02.alias("hr_02_pm10"),\
                    #df.hr_03.alias("hr_03_pm10"),\
                    #df.hr_04.alias("hr_04_pm10"),\
                    #df.hr_05.alias("hr_05_pm10"),\
                    #df.hr_06.alias("hr_06_pm10"),\
                    #df.hr_07.alias("hr_07_pm10"),\
                    #df.hr_08.alias("hr_08_pm10"),\
                    df.hr_09.alias("hr_01_pm10"),\
                    df.hr_10.alias("hr_02_pm10"),\
                    df.hr_11.alias("hr_03_pm10"),\
                    df.hr_12.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")
'''

'\ndf_pm10_12 = df.select(                    df.date.alias("datepm10"),                    df.location.alias("dateloc_pm10"),                    #df.hr_01.alias("hr_01_pm10"),                    #df.hr_02.alias("hr_02_pm10"),                    #df.hr_03.alias("hr_03_pm10"),                    #df.hr_04.alias("hr_04_pm10"),                    #df.hr_05.alias("hr_05_pm10"),                    #df.hr_06.alias("hr_06_pm10"),                    #df.hr_07.alias("hr_07_pm10"),                    #df.hr_08.alias("hr_08_pm10"),                    df.hr_09.alias("hr_01_pm10"),                    df.hr_10.alias("hr_02_pm10"),                    df.hr_11.alias("hr_03_pm10"),                    df.hr_12.alias("hr_04_pm10"),                    "measure")                    .filter(df.measure=="PM10")\n'

In [0]:
df_pm10_24 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_21.alias("hr_01_pm10"),\
                    df.hr_22.alias("hr_02_pm10"),\
                    df.hr_23.alias("hr_03_pm10"),\
                    df.hr_24.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_23 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_20.alias("hr_01_pm10"),\
                    df.hr_21.alias("hr_02_pm10"),\
                    df.hr_22.alias("hr_03_pm10"),\
                    df.hr_23.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_22 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_19.alias("hr_01_pm10"),\
                    df.hr_20.alias("hr_02_pm10"),\
                    df.hr_21.alias("hr_03_pm10"),\
                    df.hr_22.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_21 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_18.alias("hr_01_pm10"),\
                    df.hr_19.alias("hr_02_pm10"),\
                    df.hr_20.alias("hr_03_pm10"),\
                    df.hr_21.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_20 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_17.alias("hr_01_pm10"),\
                    df.hr_18.alias("hr_02_pm10"),\
                    df.hr_19.alias("hr_03_pm10"),\
                    df.hr_20.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_19 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_16.alias("hr_01_pm10"),\
                    df.hr_17.alias("hr_02_pm10"),\
                    df.hr_18.alias("hr_03_pm10"),\
                    df.hr_19.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_18 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_15.alias("hr_01_pm10"),\
                    df.hr_16.alias("hr_02_pm10"),\
                    df.hr_17.alias("hr_03_pm10"),\
                    df.hr_18.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_17 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_14.alias("hr_01_pm10"),\
                    df.hr_15.alias("hr_02_pm10"),\
                    df.hr_16.alias("hr_03_pm10"),\
                    df.hr_17.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_16 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_13.alias("hr_01_pm10"),\
                    df.hr_14.alias("hr_02_pm10"),\
                    df.hr_15.alias("hr_03_pm10"),\
                    df.hr_16.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_15 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_12.alias("hr_01_pm10"),\
                    df.hr_13.alias("hr_02_pm10"),\
                    df.hr_14.alias("hr_03_pm10"),\
                    df.hr_15.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_14 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_11.alias("hr_01_pm10"),\
                    df.hr_12.alias("hr_02_pm10"),\
                    df.hr_13.alias("hr_03_pm10"),\
                    df.hr_14.alias("hr_04_pm01"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_13 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_10.alias("hr_01_pm10"),\
                    df.hr_11.alias("hr_02_pm10"),\
                    df.hr_12.alias("hr_03_pm10"),\
                    df.hr_13.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 

df_pm10_12 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_09.alias("hr_01_pm10"),\
                    df.hr_10.alias("hr_02_pm10"),\
                    df.hr_11.alias("hr_03_pm10"),\
                    df.hr_12.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 
                    
df_pm10_11 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_08.alias("hr_01_pm10"),\
                    df.hr_09.alias("hr_02_pm10"),\
                    df.hr_10.alias("hr_03_pm10"),\
                    df.hr_11.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 

df_pm10_10 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_07.alias("hr_01_pm10"),\
                    df.hr_08.alias("hr_02_pm10"),\
                    df.hr_09.alias("hr_03_pm10"),\
                    df.hr_10.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 

df_pm10_09 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_06.alias("hr_01_pm10"),\
                    df.hr_07.alias("hr_02_pm10"),\
                    df.hr_08.alias("hr_03_pm10"),\
                    df.hr_09.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 

df_pm10_08 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_05.alias("hr_01_pm10"),\
                    df.hr_06.alias("hr_02_pm10"),\
                    df.hr_07.alias("hr_03_pm10"),\
                    df.hr_08.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 

df_pm10_07 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_04.alias("hr_01_pm10"),\
                    df.hr_05.alias("hr_02_pm10"),\
                    df.hr_06.alias("hr_03_pm10"),\
                    df.hr_07.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 

df_pm10_06 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_03.alias("hr_01_pm10"),\
                    df.hr_04.alias("hr_02_pm10"),\
                    df.hr_05.alias("hr_03_pm10"),\
                    df.hr_06.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 

df_pm10_05 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_02.alias("hr_01_pm10"),\
                    df.hr_03.alias("hr_02_pm10"),\
                    df.hr_04.alias("hr_03_pm10"),\
                    df.hr_05.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 

df_pm10_04 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_01.alias("hr_01_pm10"),\
                    df.hr_02.alias("hr_02_pm10"),\
                    df.hr_03.alias("hr_03_pm10"),\
                    df.hr_04.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 
    
df_pm10_03 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_24.alias("hr_01_pm10"),\
                    df.hr_01.alias("hr_02_pm10"),\
                    df.hr_02.alias("hr_03_pm10"),\
                    df.hr_03.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10") 

df_pm10_02 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_23.alias("hr_01_pm10"),\
                    df.hr_24.alias("hr_02_pm10"),\
                    df.hr_01.alias("hr_03_pm10"),\
                    df.hr_02.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

df_pm10_01 = df.select(\
                    df.date.alias("datepm10"),\
                    df.location.alias("dateloc_pm10"),\
                    df.hr_22.alias("hr_01_pm10"),\
                    df.hr_23.alias("hr_02_pm10"),\
                    df.hr_24.alias("hr_03_pm10"),\
                    df.hr_01.alias("hr_04_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")

In [0]:
#df_pm10_01.show(5)

In [0]:
#df_pm10_01.count()

In [0]:
#df_pm10_02.show(5)

In [0]:
#df_pm10_02.count()

In [0]:
df_pm25_24 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_21.alias("hr_01_pm25"),\
                    df.hr_22.alias("hr_02_pm25"),\
                    df.hr_23.alias("hr_03_pm25"),\
                    df.hr_24.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_23 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_20.alias("hr_01_pm25"),\
                    df.hr_21.alias("hr_02_pm25"),\
                    df.hr_22.alias("hr_03_pm25"),\
                    df.hr_23.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_22 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_19.alias("hr_01_pm25"),\
                    df.hr_20.alias("hr_02_pm25"),\
                    df.hr_21.alias("hr_03_pm25"),\
                    df.hr_22.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_21 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_18.alias("hr_01_pm25"),\
                    df.hr_19.alias("hr_02_pm25"),\
                    df.hr_20.alias("hr_03_pm25"),\
                    df.hr_21.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_20 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_17.alias("hr_01_pm25"),\
                    df.hr_18.alias("hr_02_pm25"),\
                    df.hr_19.alias("hr_03_pm25"),\
                    df.hr_20.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_19 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_16.alias("hr_01_pm25"),\
                    df.hr_17.alias("hr_02_pm25"),\
                    df.hr_18.alias("hr_03_pm25"),\
                    df.hr_19.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_18 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_15.alias("hr_01_pm25"),\
                    df.hr_16.alias("hr_02_pm25"),\
                    df.hr_17.alias("hr_03_pm25"),\
                    df.hr_18.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_17 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_14.alias("hr_01_pm25"),\
                    df.hr_15.alias("hr_02_pm25"),\
                    df.hr_16.alias("hr_03_pm25"),\
                    df.hr_17.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_16 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_13.alias("hr_01_pm25"),\
                    df.hr_14.alias("hr_02_pm25"),\
                    df.hr_15.alias("hr_03_pm25"),\
                    df.hr_16.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_15 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_12.alias("hr_01_pm25"),\
                    df.hr_13.alias("hr_02_pm25"),\
                    df.hr_14.alias("hr_03_pm25"),\
                    df.hr_15.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_14 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_11.alias("hr_01_pm25"),\
                    df.hr_12.alias("hr_02_pm25"),\
                    df.hr_13.alias("hr_03_pm25"),\
                    df.hr_14.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_13 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_10.alias("hr_01_pm25"),\
                    df.hr_11.alias("hr_02_pm25"),\
                    df.hr_12.alias("hr_03_pm25"),\
                    df.hr_13.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 

df_pm25_12 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_09.alias("hr_01_pm25"),\
                    df.hr_10.alias("hr_02_pm25"),\
                    df.hr_11.alias("hr_03_pm25"),\
                    df.hr_12.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 
                    
df_pm25_11 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_08.alias("hr_01_pm25"),\
                    df.hr_09.alias("hr_02_pm25"),\
                    df.hr_10.alias("hr_03_pm25"),\
                    df.hr_11.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 

df_pm25_10 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_07.alias("hr_01_pm25"),\
                    df.hr_08.alias("hr_02_pm25"),\
                    df.hr_09.alias("hr_03_pm25"),\
                    df.hr_10.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 

df_pm25_09 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_06.alias("hr_01_pm25"),\
                    df.hr_07.alias("hr_02_pm25"),\
                    df.hr_08.alias("hr_03_pm25"),\
                    df.hr_09.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 

df_pm25_08 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_05.alias("hr_01_pm25"),\
                    df.hr_06.alias("hr_02_pm25"),\
                    df.hr_07.alias("hr_03_pm25"),\
                    df.hr_08.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 

df_pm25_07 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_04.alias("hr_01_pm25"),\
                    df.hr_05.alias("hr_02_pm25"),\
                    df.hr_06.alias("hr_03_pm25"),\
                    df.hr_07.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 

df_pm25_06 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_03.alias("hr_01_pm25"),\
                    df.hr_04.alias("hr_02_pm25"),\
                    df.hr_05.alias("hr_03_pm25"),\
                    df.hr_06.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 

df_pm25_05 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_02.alias("hr_01_pm25"),\
                    df.hr_03.alias("hr_02_pm25"),\
                    df.hr_04.alias("hr_03_pm25"),\
                    df.hr_05.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 

df_pm25_04 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_01.alias("hr_01_pm25"),\
                    df.hr_02.alias("hr_02_pm25"),\
                    df.hr_03.alias("hr_03_pm25"),\
                    df.hr_04.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 
    
df_pm25_03 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_24.alias("hr_01_pm25"),\
                    df.hr_01.alias("hr_02_pm25"),\
                    df.hr_02.alias("hr_03_pm25"),\
                    df.hr_03.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 

df_pm25_02 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_23.alias("hr_01_pm25"),\
                    df.hr_24.alias("hr_02_pm25"),\
                    df.hr_01.alias("hr_03_pm25"),\
                    df.hr_02.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

df_pm25_01 = df.select(\
                    df.date.alias("datepm25"),\
                    df.location.alias("dateloc_pm25"),\
                    df.hr_22.alias("hr_01_pm25"),\
                    df.hr_23.alias("hr_02_pm25"),\
                    df.hr_24.alias("hr_03_pm25"),\
                    df.hr_01.alias("hr_04_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5")

In [0]:
#df_pm25_11.show(5)

In [0]:
#df_pm25.show(5)

In [0]:
'''
df_AMB_TEMP = df.select(\
                    df.date.alias("dateAMB_TEMP"),\
                    df.location.alias("dateloc_AMB_TEMP"),\
                    #df.hr_01.alias("hr_01_AMB_TEMP"),\
                    #df.hr_02.alias("hr_02_AMB_TEMP"),\
                    #df.hr_03.alias("hr_03_AMB_TEMP"),\
                    #df.hr_04.alias("hr_04_AMB_TEMP"),\
                    #df.hr_05.alias("hr_05_AMB_TEMP"),\
                    #df.hr_06.alias("hr_06_AMB_TEMP"),\
                    #df.hr_07.alias("hr_07_AMB_TEMP"),\
                    #df.hr_08.alias("hr_08_AMB_TEMP"),\
                    df.hr_09.alias("hr_09_AMB_TEMP"),\
                    df.hr_10.alias("hr_10_AMB_TEMP"),\
                    df.hr_11.alias("hr_11_AMB_TEMP"),\
                    df.hr_12.alias("hr_12_AMB_TEMP"),\
                    "measure")\
                    .filter(df.measure=="AMB_TEMP") 
'''

'\ndf_AMB_TEMP = df.select(                    df.date.alias("dateAMB_TEMP"),                    df.location.alias("dateloc_AMB_TEMP"),                    #df.hr_01.alias("hr_01_AMB_TEMP"),                    #df.hr_02.alias("hr_02_AMB_TEMP"),                    #df.hr_03.alias("hr_03_AMB_TEMP"),                    #df.hr_04.alias("hr_04_AMB_TEMP"),                    #df.hr_05.alias("hr_05_AMB_TEMP"),                    #df.hr_06.alias("hr_06_AMB_TEMP"),                    #df.hr_07.alias("hr_07_AMB_TEMP"),                    #df.hr_08.alias("hr_08_AMB_TEMP"),                    df.hr_09.alias("hr_09_AMB_TEMP"),                    df.hr_10.alias("hr_10_AMB_TEMP"),                    df.hr_11.alias("hr_11_AMB_TEMP"),                    df.hr_12.alias("hr_12_AMB_TEMP"),                    "measure")                    .filter(df.measure=="AMB_TEMP") \n'

In [0]:
#df_AMB_TEMP.show(5)

In [0]:
'''
traing_data = df_pm25\
    .join(df_pm10, (df_pm25.datepm25==df_pm10.datepm10) & (df_pm25.dateloc_pm25==df_pm10.dateloc_pm10))\
    .join(df_AMB_TEMP, (df_pm25.datepm25==df_AMB_TEMP.dateAMB_TEMP) & (df_pm25.dateloc_pm25==df_AMB_TEMP.dateloc_AMB_TEMP))\
    .drop("dateAMB_TEMP")\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\
    .drop("dateloc_AMB_TEMP")
    
'''

'\ntraing_data = df_pm25    .join(df_pm10, (df_pm25.datepm25==df_pm10.datepm10) & (df_pm25.dateloc_pm25==df_pm10.dateloc_pm10))    .join(df_AMB_TEMP, (df_pm25.datepm25==df_AMB_TEMP.dateAMB_TEMP) & (df_pm25.dateloc_pm25==df_AMB_TEMP.dateloc_AMB_TEMP))    .drop("dateAMB_TEMP")    .drop("measure")    .drop("datepm10")    .drop("dateloc_pm25")    .drop("dateloc_pm10")    .drop("dateloc_AMB_TEMP")\n    \n'

In [0]:
traing_data24 = df_pm25_24\
    .join(df_pm10_24, (df_pm25_24.datepm25==df_pm10_24.datepm10) & (df_pm25_24.dateloc_pm25==df_pm10_24.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data23 = df_pm25_23\
    .join(df_pm10_23, (df_pm25_23.datepm25==df_pm10_23.datepm10) & (df_pm25_23.dateloc_pm25==df_pm10_23.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data22 = df_pm25_22\
    .join(df_pm10_22, (df_pm25_22.datepm25==df_pm10_22.datepm10) & (df_pm25_22.dateloc_pm25==df_pm10_22.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data21 = df_pm25_21\
    .join(df_pm10_21, (df_pm25_21.datepm25==df_pm10_21.datepm10) & (df_pm25_21.dateloc_pm25==df_pm10_21.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data20 = df_pm25_20\
    .join(df_pm10_20, (df_pm25_20.datepm25==df_pm10_20.datepm10) & (df_pm25_20.dateloc_pm25==df_pm10_20.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data19 = df_pm25_19\
    .join(df_pm10_19, (df_pm25_19.datepm25==df_pm10_19.datepm10) & (df_pm25_19.dateloc_pm25==df_pm10_19.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data18 = df_pm25_18\
    .join(df_pm10_18, (df_pm25_18.datepm25==df_pm10_18.datepm10) & (df_pm25_18.dateloc_pm25==df_pm10_18.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data17 = df_pm25_17\
    .join(df_pm10_17, (df_pm25_17.datepm25==df_pm10_17.datepm10) & (df_pm25_17.dateloc_pm25==df_pm10_17.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data16 = df_pm25_16\
    .join(df_pm10_16, (df_pm25_16.datepm25==df_pm10_16.datepm10) & (df_pm25_16.dateloc_pm25==df_pm10_16.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data15 = df_pm25_15\
    .join(df_pm10_15, (df_pm25_15.datepm25==df_pm10_15.datepm10) & (df_pm25_15.dateloc_pm25==df_pm10_15.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data14 = df_pm25_14\
    .join(df_pm10_14, (df_pm25_14.datepm25==df_pm10_14.datepm10) & (df_pm25_14.dateloc_pm25==df_pm10_14.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data13 = df_pm25_13\
    .join(df_pm10_13, (df_pm25_13.datepm25==df_pm10_13.datepm10) & (df_pm25_13.dateloc_pm25==df_pm10_13.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data12 = df_pm25_12\
    .join(df_pm10_12, (df_pm25_12.datepm25==df_pm10_12.datepm10) & (df_pm25_12.dateloc_pm25==df_pm10_12.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data11 = df_pm25_11\
    .join(df_pm10_11, (df_pm25_11.datepm25==df_pm10_11.datepm10) & (df_pm25_11.dateloc_pm25==df_pm10_11.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data10= df_pm25_10\
    .join(df_pm10_10, (df_pm25_10.datepm25==df_pm10_10.datepm10) & (df_pm25_10.dateloc_pm25==df_pm10_10.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data09 = df_pm25_09\
    .join(df_pm10_09, (df_pm25_09.datepm25==df_pm10_09.datepm10) & (df_pm25_09.dateloc_pm25==df_pm10_09.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data08 = df_pm25_08\
    .join(df_pm10_08, (df_pm25_08.datepm25==df_pm10_08.datepm10) & (df_pm25_08.dateloc_pm25==df_pm10_08.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

traing_data07 = df_pm25_07\
    .join(df_pm10_07, (df_pm25_07.datepm25==df_pm10_07.datepm10) & (df_pm25_07.dateloc_pm25==df_pm10_07.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\
    
traing_data06 = df_pm25_06\
    .join(df_pm10_06, (df_pm25_06.datepm25==df_pm10_06.datepm10) & (df_pm25_06.dateloc_pm25==df_pm10_06.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\
        
traing_data05 = df_pm25_05\
    .join(df_pm10_05, (df_pm25_05.datepm25==df_pm10_05.datepm10) & (df_pm25_05.dateloc_pm25==df_pm10_05.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\
        
traing_data04 = df_pm25_04\
    .join(df_pm10_04, (df_pm25_04.datepm25==df_pm10_04.datepm10) & (df_pm25_04.dateloc_pm25==df_pm10_04.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\
        
traing_data03 = df_pm25_03\
    .join(df_pm10_03, (df_pm25_03.datepm25==df_pm10_03.datepm10) & (df_pm25_03.dateloc_pm25==df_pm10_03.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\
        
traing_data02 = df_pm25_02\
    .join(df_pm10_02, (df_pm25_02.datepm25==df_pm10_02.datepm10) & (df_pm25_02.dateloc_pm25==df_pm10_02.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\
        
traing_data01 = df_pm25_01\
    .join(df_pm10_01, (df_pm25_01.datepm25==df_pm10_01.datepm10) & (df_pm25_01.dateloc_pm25==df_pm10_01.dateloc_pm10))\
    .drop("measure")\
    .drop("datepm10")\
    .drop("dateloc_pm25")\
    .drop("dateloc_pm10")\

In [0]:
traing_data01 = traing_data01.union(traing_data02)
traing_data01 = traing_data01.union(traing_data03)
traing_data01 = traing_data01.union(traing_data04)
traing_data01 = traing_data01.union(traing_data05)
traing_data01 = traing_data01.union(traing_data06)
traing_data01 = traing_data01.union(traing_data07)
traing_data01 = traing_data01.union(traing_data08)
traing_data01 = traing_data01.union(traing_data09)
traing_data01 = traing_data01.union(traing_data10)
traing_data01 = traing_data01.union(traing_data11)
traing_data01 = traing_data01.union(traing_data12)
traing_data01 = traing_data01.union(traing_data13)
traing_data01 = traing_data01.union(traing_data14)
traing_data01 = traing_data01.union(traing_data15)
traing_data01 = traing_data01.union(traing_data16)
traing_data01 = traing_data01.union(traing_data17)
traing_data01 = traing_data01.union(traing_data18)
traing_data01 = traing_data01.union(traing_data19)
traing_data01 = traing_data01.union(traing_data20)
traing_data01 = traing_data01.union(traing_data21)
traing_data01 = traing_data01.union(traing_data22)
traing_data01 = traing_data01.union(traing_data23)
traing_data01 = traing_data01.union(traing_data24)


In [0]:
#traing_data01.count()

In [0]:
#traing_data01.show(5)

In [0]:
#traing_data.corr("hr_09_NO2", "hr_09_")

In [0]:
from pyspark.sql import functions as F
formulated_traning_data = traing_data01\
            .select("*", F.when(traing_data01.hr_04_pm25 >= 50, 1).otherwise(0))\
            .drop("hr_04_pm10")\
            .drop("hr_04_pm25")\
            .drop("datepm25")

In [0]:
#formulated_traning_data.show(5)

In [0]:
#formulated_traning_data.show(5)

In [0]:
from pyspark.sql import functions as F
formulated_traning_data = traing_data01\
            .select("*", F.when(traing_data01.hr_04_pm25 >= 50, 1).otherwise(0))\
            .withColumnRenamed("CASE WHEN (hr_04_pm25 >= 50) THEN 1 ELSE 0 END", "pm25_condiction")\
            .drop("hr_04_pm10")\
            .drop("hr_04_pm25")\
            .drop("datepm25")

In [0]:
#formulated_traning_data.show(5)

In [0]:
#formulated_traning_data.show(5)

In [0]:
#formulated_traning_data.show(5)

# 預處理Model

In [0]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils


In [0]:
#r.hr_01_pm25, r.hr_02_pm25, r.hr_03_pm25, r.hr_04_pm25, r.hr_05_pm25, r.hr_06_pm25, r.hr_07_pm25, r.hr_08_pm25, 
#r.hr_01_pm10, r.hr_02_pm10, r.hr_03_pm10, r.hr_04_pm10, r.hr_05_pm10, r.hr_06_pm10, r.hr_07_pm10, r.hr_08_pm10, 
#r.hr_01_AMB_TEMP, r.hr_02_AMB_TEMP, r.hr_03_AMB_TEMP, r.hr_04_AMB_TEMP, r.hr_05_AMB_TEMP, r.hr_06_AMB_TEMP, r.hr_07_AMB_TEMP, r.hr_08_AMB_TEMP, r.hr_09_AMB_TEMP, r.hr_10_AMB_TEMP, r.hr_11_AMB_TEMP 

In [0]:
from pyspark.mllib.regression import LabeledPoint

LabelPoints = formulated_traning_data.rdd\
            .map(lambda r: LabeledPoint(r.pm25_condiction, \
                                        [r.hr_01_pm25, r.hr_02_pm25, r.hr_03_pm25, \
                                         r.hr_01_pm10, r.hr_02_pm10, r.hr_03_pm10]))


In [0]:
(trainData,validationData,testData) = LabelPoints.randomSplit([10,0,0]) 

In [0]:
#LabelPoints.count()

In [0]:
#trainData.count()

# Decision Tree Model

In [0]:
'''
DTModel10 = DecisionTree.trainClassifier(trainData,
        numClasses=2,
        categoricalFeaturesInfo={},
        impurity="gini",
        maxDepth=10,
        maxBins=32)
'''

'\nDTModel10 = DecisionTree.trainClassifier(trainData,\n        numClasses=2,\n        categoricalFeaturesInfo={},\n        impurity="gini",\n        maxDepth=10,\n        maxBins=32)\n'

In [0]:
'''
LabelPoints = formulated_traning_data.rdd\
            .map(lambda r: LabeledPoint(r.pm25_condiction, \
                                        [r.hr_08_pm25, r.hr_09_pm25, r.hr_10_pm25, \
                                         r.hr_08_pm10, r.hr_09_pm10, r.hr_10_pm10]))
(trainData,validationData,testData) = LabelPoints.randomSplit([10,0,0]) 
'''

'\nLabelPoints = formulated_traning_data.rdd            .map(lambda r: LabeledPoint(r.pm25_condiction,                                         [r.hr_08_pm25, r.hr_09_pm25, r.hr_10_pm25,                                          r.hr_08_pm10, r.hr_09_pm10, r.hr_10_pm10]))\n(trainData,validationData,testData) = LabelPoints.randomSplit([10,0,0]) \n'

In [0]:
# Save model
#DTModel10.save(sc, "content/drive/My Drive/colab/model/DTModel10")

# Random Forest Model

In [0]:
'''
LabelPoints = formulated_traning_data.rdd\
            .map(lambda r: LabeledPoint(r.pm25_condiction, \
                                        [r.hr_04_pm25, r.hr_05_pm25, r.hr_06_pm25, \
                                         r.hr_04_pm10, r.hr_05_pm10, r.hr_06_pm10]))
(trainData,validationData,testData) = LabelPoints.randomSplit([10,0,0]) 
'''

'\nLabelPoints = formulated_traning_data.rdd            .map(lambda r: LabeledPoint(r.pm25_condiction,                                         [r.hr_04_pm25, r.hr_05_pm25, r.hr_06_pm25,                                          r.hr_04_pm10, r.hr_05_pm10, r.hr_06_pm10]))\n(trainData,validationData,testData) = LabelPoints.randomSplit([10,0,0]) \n'

In [0]:
RFModel40 = RandomForest.trainClassifier(trainData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=10, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=10, maxBins=5)

In [0]:
# Save model
RFModel40.save(sc, "content/drive/My Drive/colab/model/RFModel40")

# Regression

In [0]:
#regression_training = \
#             traing_data.drop("hr_12_pm10")\
#            .drop("datepm25")


In [0]:
#from pyspark.mllib.regression import LabeledPoint

#LabelPoints = regression_training.rdd\
#            .map(lambda r: LabeledPoint(r.hr_12_pm25, \
#                                        [r.hr_01_pm25, r.hr_02_pm25, r.hr_03_pm25, r.hr_04_pm25, r.hr_05_pm25, r.hr_06_pm25, r.hr_07_pm25, r.hr_08_pm25, r.hr_09_pm25, r.hr_10_pm25, r.hr_11_pm25, \
#                                         r.hr_01_pm10, r.hr_02_pm10, r.hr_03_pm10, r.hr_04_pm10, r.hr_05_pm10, r.hr_06_pm10, r.hr_07_pm10, r.hr_08_pm10, r.hr_09_pm10, r.hr_10_pm10, r.hr_11_pm10]))

In [0]:
#(trainData,validationData,testData) = LabelPoints.randomSplit([10,0,0])

In [0]:
#REModel10 = DecisionTree.trainRegressor(trainData, categoricalFeaturesInfo={},
#                                    impurity='variance', maxDepth=5, maxBins=32)

In [0]:
# Save model
#REModel10.save(sc, "content/drive/My Drive/colab/model/REModel10")

# 測資處理

In [0]:
#!wget "https://www.dropbox.com/s/3tejnx4i3v2fv0v/pm%2025%20test%20samples.zip?dl=0"

In [0]:
#!unzip 'pm 25 test samples.zip?dl=0'

In [0]:
#!ls

In [0]:
!ls /content/drive/My\ Drive/colab/pm25_test/

In [0]:
def vote(choose):
  #----
  #if sum(choose) >= 2:
  #  ans = 1
  #else:
  #  ans = 0
  #----
  #-------
  #if sum(choose) == 2:
  #  ans = choose[2]
  #elif choose[0]==choose[1] and choose[1]!=choose[2]:
  #  ans = choose[2]
  #elif sum(choose) == 3:
  #  ans = 1
  #else:
  #  ans = 0
  #-------
  if choose[1]==1:
    ans = 1
  elif choose[1]==0:
    ans = 0
  return ans 

#def strip_symbol(x):
 # for i in range(len(x)):
 #     #if x[i]==0:
 #     #  x[i] = "0"
 #     #x[i] = x[i].strip("-*#x") # remove non-digits
 #     if x[i] == "": x[i] = ""
 #     if "#" in x[i]:x[i] = "" 
 #     if "*" in x[i]:x[i] = ""
 #     if "x" in x[i]:x[i] = ""
#      if "-" in x[i]:x[i] = ""
#  return x
def strip_symbol(x):
  for i in range(len(x)):
      x[i] = x[i].strip("-*#x") # remove non-digits

  return x

def del2big(list_in):
  for i in list_in:
    if i != "":
      if int(i) > 500:
        t = list_in.index(i)
        list_in[t] = ""
      elif int(i) < 0:
        t = list_in.index(i)
        list_in[t] = ""

  return list_in

def fillin(list_in):
  #print("init print in: ",str(list_in),'\n')
  count_Notnull = 0
  tmp_list = list_in.copy()
  
  #print("init print tmp: ",str(tmp_list),'\n')
  avg = 0
  for i in list_in:
    if i != '':
      count_Notnull += 1
  
  for i in range(len(tmp_list)):
    if tmp_list[i] == '':
      tmp_list[i] = '0'

  #print("1stend print in: ",str(list_in),'\n')
  #print("end print tmp: ",str(tmp_list),'\n')
  #b = [i for i in range(len(a)) if a[i] == '0']

  tmp_list = [int(i) for i in tmp_list]
  avg = int(sum(tmp_list)/count_Notnull)
  #print('avg: ',avg)

  for t in range(len(list_in)):
    if list_in[t] == '':
      list_in[t] = str(avg)
      #print('tmp:: ' + list_in[t])

  #print("end print in: ",str(list_in),'\n')

  return list_in

#       predict
#[a,a,a,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24] len=27
                                                            #-3 -2 -1
                                                                  #(i +26 -1)
def getdata(data, list_in, pre, count):
  for index in range(pre-25, -25, -1):
    if data[index] != "" and count != 3:
      list_in.append(data[index])
      count += 1

  return list_in, count

def getYesdata(data, list_in, pre, count):
  for index in range(-1, -25, -1):
    if data[index] != "" and count != 3:
      list_in.append(data[index])
      count += 1

  return list_in, count

In [0]:
'''
if t == 0:
        list_in[t] = str( int( round( (int(list_in[t+1])+int(list_in[t+2])) / 2, 0) ) )
        print('t','0',':')
        print(list_in[0])
      elif t == 1:
        list_in[t] = str( int( round( (int(list_in[t+1])+int(list_in[t-1])) / 2, 0) ) )
        print('t','1',':')
        print(list_in[1])
      elif t == 2:
        list_in[t] = str( int( round( (int(list_in[t-1])+int(list_in[t-2])) / 2, 0) ) )
        print('t','2',':')
        print(list_in[2])
      elif t == 3:
        list_in[t] = str( int( round( (int(list_in[t-1])+int(list_in[t-2])) / 2, 0) ) )
        print('t','3',':')
        print(list_in[3])
      elif t == 4:
        list_in[t] = str( int( round( (int(list_in[t-1])+int(list_in[t-2])) / 2, 0) ) )
        print('t','4',':')
        print(list_in[4])
      elif t == 5:
        #list_in[t] = str( int( round( (int(list_in[t-1])+int(list_in[t-2])) / 2, 0) ) )
        list_in[t] = str(avg)
        print('t','5',':')
        print(list_in[5])
'''

In [0]:
from pyspark.mllib.regression import LabeledPoint
import time
tt_data = []
ans = "["
total_test = 100

for i in range(total_test):
  num = i+1
  print('Round: ',num)
  if num < 10:
    file_num = '/content/drive/My Drive/colab/pm25_test/00' + str(num) + '.csv'
    #print(1)
  elif num >= 10 and num < 100:
    file_num = '/content/drive/My Drive/colab/pm25_test/0' + str(num) + '.csv'
    #print(2)
  else:
    file_num = '/content/drive/My Drive/colab/pm25_test/' + str(num) + '.csv'
    #print(3)
  print(file_num)
  # get start time
  start = time.time()

  # read file
  test_data = sc.textFile(file_num)
  #print(test_data.count())
  #test_data = test_data.map(strip_symbol)
  test_data_rdd = test_data.map(lambda line : line.split(","))
  test_data_rdd = test_data_rdd.filter(lambda x: x[2] == 'PM2.5' or x[2] == 'PM10')


  # get list
  tt_data = test_data_rdd.collect()
  # print(tt_data)
  ttt_data = tt_data[-1]      # pm25 this day
  ttt_pm10_data = tt_data[-2] # pm10 this day
  #ttt_temp_data = tt_data[-3]  # no2  this day

  ttt_data_yes = tt_data[-4]
  ttt_pm10_yes = tt_data[-5]
  #ttt_temp_yes = tt_data[-6]
  print(ttt_data)
  print(ttt_pm10_data)
  #print(ttt_temp_data)
  print(ttt_data_yes)
  print(ttt_pm10_yes)
  #print(ttt_temp_yes)
  predict = 1
  for i in range(-1,-25,-1):
    if ttt_data[i] != '':
      print("i: ",i)
      predict = (i + 26 - 1)
      break
  print("Predict: ",predict)

  #----------------------------------------------------------------------
  # new todo
  print("============= start =============")
  only1data = []
  onlypm10data = []
  #onlytempdata = []

  count25 = 0
  count10 = 0
  count2 = 0
  ###################################################
  only1data, count25 = getdata(ttt_data, only1data, predict, count25)
  onlypm10data, count10 = getdata(ttt_pm10_data, onlypm10data, predict, count10)
  #onlytempdata, count2 = getdata(ttt_temp_data, onlytempdata, predict, count2)
  ##print("catch today hours")
  ##print(only1data, ' ', count25)
  ##print(onlypm10data, ' ', count10)
  ##print(onlytempdata, ' ', count2)
  ###################################################
  print("catch yes hours")
  if count25 != 3:
    only1data, count25 = getYesdata(ttt_data_yes, only1data, predict, count25)
  if count10 != 3: 
    onlypm10data, count10 = getYesdata(ttt_pm10_yes, onlypm10data, predict, count10)
  #if count2 != 3: 
  #  onlytempdata, count2 = getYesdata(ttt_temp_yes, onlytempdata, predict, count2)
  only1data.reverse()
  onlypm10data.reverse()
  #onlytempdata.reverse()
  

  ##print("catch 3 hours")
  ##print(only1data)
  ##print(onlypm10data)
  ##print(onlytempdata)
  ##print('finish test')

  ##print('\nafter strip')
  strip_symbol(only1data)
  strip_symbol(onlypm10data)
  #strip_symbol(onlytempdata)
  ##print(only1data)
  ##print(onlypm10data)
  ##print(onlytempdata)

  ##print('\nafter del')
  only1data = del2big(only1data)
  onlypm10data = del2big(onlypm10data)
  #onlytempdata = del2big(onlytempdata)
  ##print(only1data)
  ##print(onlypm10data)
  ##print(onlytempdata)

  ##print('\nafter fillin')
  only1data = fillin(only1data)
  onlypm10data = fillin(onlypm10data)
  #onlytempdata = fillin(onlytempdata)
  ##print(only1data)
  ##print(onlypm10data)
  ##print(onlytempdata)

  print('\nafter clean')
  print(only1data)
  print(onlypm10data)
  #print(onlytempdata)
  only1data.extend(onlypm10data)
  #only1data.extend(onlytempdata)
  only1data_rdd = sc.parallelize(only1data)
  only1data_rdd = only1data_rdd.map(lambda x: float(x))
  only1data_rdd.take(6)
  ##print('\n')
  #----------------------------------------------------------------------    
  

  
  only1_data = LabeledPoint(1,only1data)
  only1_data

  # Decision Tree Model
  #Load_DTModel = DecisionTreeModel.load(sc, "content/drive/My Drive/colab/model/DTModel10")
  #prediction_DT = Load_DTModel.predict(only1_data.features)
  #print('Prediction_DT: ', int(prediction_DT))

  # Random Forest Model
  Load_RFModel = RandomForestModel.load(sc, "content/drive/My Drive/colab/model/RFModel40")
  prediction_RF = Load_RFModel.predict(only1_data.features)
  print('Prediction_RF: ', int(prediction_RF))

  # Regression Model
  #Load_REModel = DecisionTreeModel.load(sc, "content/drive/My Drive/colab/model/REModel10")
  #prediction_data = Load_REModel.predict(only1_data.features)
  #print('Regression Model: ', prediction_data)

  # determine while >= 50
  #if(prediction_data>=50):
  #  prediction_RE=1
  #else:
  #  prediction_RE=0
  #print('Prediction_RE: ', prediction_RE)

  choose=[]
  choose.append(0)
  #choose.append(int(prediction_DT))
  choose.append(int(prediction_RF))
  choose.append(0)
  #choose.append(prediction_RE)
  print('Choose result: ', choose)
  # vote result
  vote_result = vote(choose)
  print('Vote result: ', vote(choose))
  
  # update ans
  '''
  ans += str(vote_result)
  if num != total_test:
    ans += ","
  else:
    ans += "]"
'''
  # " "
  ans += "\""

  ans += str(vote_result)
  if num != total_test:
    ans += "\", "
  else:
    ans += "\"]"

  # get end time
  end = time.time()
  print('Period: ' + str(round(end-start,2)) + 's')
  print("\n\n")
  print("Answer: ",ans)
print("\n##############################################################################\n")
print("Answer: ",ans)

Round:  1
/content/drive/My Drive/colab/pm25_test/001.csv
['2015/05/11', '龍潭', 'PM2.5', '33', '34', '29', '17', '16', '26', '33', '33', '30', '29', '27', '22', '33', '40', '53', '', '', '', '', '', '', '', '', '']
['2015/05/11', '龍潭', 'PM10', '75', '76', '77', '70', '74', '95', '91', '80', '46', '50', '47', '52', '66', '81', '107', '112', '104', '86', '80', '83', '92', '83', '76', '56']
['2015/05/10', '龍潭', 'PM10', '48', '56', '57', '49', '40', '41', '39', '50', '55', '58', '44', '37', '32', '32', '31', '37', '48', '58', '62', '66', '63', '72', '75', '77']
['2015/05/09', '龍潭', 'PM2.5', '21', '21', '21', '15', '15', '11', '15', '13', '22', '21', '23', '20', '25', '24', '18', '15', '17', '23', '22', '19', '19', '18', '24', '17']
i:  -10
Predict:  15
============= start =============
catch yes hours

after clean
['33', '40', '53']
['66', '81', '107']
Prediction_RF:  0
Choose result:  [0, 0, 0]
Vote result:  0
Period: 2.99s



Answer:  ["0", 
Round:  2
/content/drive/My Drive/colab/pm25_te

In [0]:
 #["0","1","0","0","0","1","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","1","0","0","0","0","1","0","1","1","1","1","0","1","0","1","1","0","1","0","0","1","1","1","0","0","1","0","0","0","0","0","1","0","0","0","1","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","1","0","0","0","0","1","0","1","1","1","1","0","1","0","1","1","0","1","0","0","1","1","1","0","0","1","0","0","0"]

In [0]:
#["0","1","1","0","1","1","0","0","0","0","0","0","0","0","1","0","1","0","0","1","0","1","1","0","0","0","1","0","1","1","1","1","0","1","0","1","1","0","1","0","0","1","1","1","0","1","1","1","1","1","0","1","0","0","1","1","0","0","0","0","1","0","1","1","0","0","1","0","1","1","0","1","0","0","0","1","0","1","1","0","0","1","1","0","1","1","1","0","0","0","1","1","0","1","0","1","1","1","1","1"]

In [0]:
ans = ["0", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "0", "1", "0", "0", "0", "0", "0", "1", "1", "0", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "0", "1", "0", "1", "0", "1", "0", "0", "0", "1", "1", "0", "1", "1", "0", "0", "0", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1"]
# 3 model : 64
#our = ["0","1","0","0","1","1","0","0","0","0","0","0","0","0","1","0","1","0","0","1","0","1","1","0","0","0","1","0","1","1","1","1","0","1","0","1","1","0","1","0","0","1","1","0","0","1","1","1","1","1","0","1","0","0","1","1","0","0","0","0","1","0","1","1","0","0","1","0","1","1","0","1","0","0","0","1","0","1","1","0","0","1","1","0","1","0","1","0","0","0","1","1","0","0","0","1","1","1","1","1"]
# only regression : 65
#our = ["0","0","1","0","0","0","0","0","0","0","0","0","0","0","1","0","1","0","0","1","0","0","0","0","0","0","0","0","0","1","0","0","0","1","0","1","1","0","0","1","0","0","1","0","1","1","0","0","1","1","0","0","0","0","1","1","0","0","0","1","0","0","0","1","0","0","1","0","1","0","0","1","0","0","0","0","0","0","0","0","0","0","0","0","0","1","0","0","1","0","1","1","0","0","0","0","1","1","0","0"]
# only DT : 67
#our = ["0","1","0","0","1","1","0","0","0","0","0","0","0","0","1","0","1","0","0","1","0","1","1","0","0","0","1","0","1","1","1","1","0","1","0","1","1","0","1","0","0","1","1","1","0","1","1","1","1","1","0","1","0","0","1","0","0","0","0","0","1","0","1","1","0","0","1","0","1","1","0","1","0","0","0","1","0","1","1","0","0","1","1","0","1","0","1","0","0","1","1","1","0","0","0","1","1","1","1","1"]
# only RF : 63
#our = ["0","1","0","0","1","1","0","0","0","0","0","0","0","0","1","0","1","0","0","1","0","1","1","0","0","1","1","0","1","1","1","1","0","1","0","1","1","0","1","0","0","1","1","0","0","1","1","1","1","1","0","1","0","0","1","1","0","0","0","0","1","0","1","1","0","0","1","0","1","1","0","1","0","0","0","1","0","1","1","0","0","1","1","0","1","0","1","0","0","0","1","1","0","0","0","1","1","1","1","1"]
# forget ?
#our = ["0","1","0","0","1","1","0","0","0","0","0","0","0","0","1","0","1","0","0","1","0","1","1","0","0","0","1","0","1","1","1","1","0","1","0","1","1","0","1","0","0","1","1","0","0","1","1","1","1","1","0","1","0","0","1","1","0","0","0","0","1","0","1","1","0","0","1","0","1","1","0","1","0","0","0","1","0","1","1","0","0","1","1","0","1","0","1","0","0","0","1","1","0","0","0","1","1","1","1","1"]
# new v1 : 59
#our = ["0","0","0","0","1","0","0","0","0","0","0","1","0","1","0","0","0","0","0","0","0","1","1","0","0","0","0","0","0","1","0","1","0","0","0","1","0","0","1","0","0","0","0","0","0","0","1","0","0","0","0","0","0","0","1","0","0","0","1","0","1","0","1","1","1","0","1","0","1","0","1","0","0","0","0","1","0","0","1","0","0","0","1","0","0","0","1","0","0","0","0","0","0","0","1","1","1","0","0","1"]
# new v2 : 68(?)
#our = ["0","1","0","0","1","1","0","0","0","0","0","0","0","0","1","0","1","0","0","1","0","1","1","0","0","0","1","0","1","1","1","1","0","1","0","1","1","0","1","0","0","1","1","0","0","1","1","1","1","1","0","0","0","0","1","1","0","0","0","0","1","0","1","1","0","0","1","0","1","0","0","1","0","0","0","1","0","1","1","0","0","1","1","0","1","0","1","0","0","0","1","1","1","1","0","1","1","1","1","1"]
# new v3 : 66 (test vote)
#our = ["0","0","0","0","0","0","0","0","0","0","0","0","0","0","1","0","0","0","0","0","0","0","0","0","0","0","0","0","0","1","0","0","0","1","0","1","1","0","0","0","0","0","1","0","0","1","0","1","1","1","0","0","0","0","1","1","0","0","0","0","0","0","0","1","0","0","1","0","1","0","0","1","0","0","0","0","0","0","0","0","0","0","0","0","0","1","1","0","0","0","1","0","0","1","0","0","1","0","1","0"]
# new v4 : 68 (original vote)
#our = ["0","1","0","0","1","1","0","0","0","0","0","0","0","0","1","0","1","0","0","1","0","1","1","0","0","0","1","0","1","1","1","1","0","1","0","1","1","0","1","0","0","1","1","1","0","1","1","1","1","1","0","0","0","0","1","1","0","0","0","0","1","0","1","1","0","0","1","0","1","0","0","1","0","0","0","1","0","1","1","0","0","1","1","0","1","1","1","0","0","0","1","1","1","1","0","1","1","1","1","1"]
# new v5 : 69 (test vote)
#our = ["0","0","1","0","0","0","0","0","0","0","0","0","0","0","1","0","0","0","0","0","0","0","0","0","0","0","0","0","0","1","0","0","0","1","0","1","1","0","0","1","0","0","1","0","1","1","0","1","1","1","0","0","0","0","1","1","0","0","0","1","0","0","0","1","0","0","1","0","1","0","0","1","0","0","0","0","0","0","0","0","0","0","0","0","0","1","1","0","1","0","1","0","0","1","0","0","1","0","1","0"]
#-------
#ans = ["0", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "0", "1", "0", "0", "0", "0", "0", "1", "1", "0", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "0", "1", "0", "1", "0", "1", "0", "0", "0", "1", "1", "0", "1", "1", "0", "0", "0", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1"]
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
#-------
# new v6 : 69 (predict+1)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v7 : 69 (6 model)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v8 : 69 (2 model)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v9 : 69 (2 model: RE:dep20)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v10 : 69 (2 model: RE:dep5)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v11 : 69 (2 model: dep5,bin32)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v12 : 69 (2 model: RF:num10;dep5,bin32)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v13 : 69 (2 model: gini RF:num10;dep5,bin32)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v14 : 69
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v15 : 69 (origin)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v16 : 69 (10:0:0=>5:5:0)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v17 : 69 (6 model)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v18 : 69 (8 model)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v19 : 70 (8 model, orgin vote)
#our = ["0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "1", "1", "0", "0", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "1", "1", "0", "1", "1", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "1", "0"]
# new v20 : 69 (RE model vaildation)
#our = ["0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v21 : 69 (RE model traindata same twice)
#our = ["0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v22 : 69 (get traindata twice)
#our = ["0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v23 : 68 (???????)
#our = ["0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "0", "0", "1", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "0", "1", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "1", "1", "0", "0", "1", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "1", "1", "0"]
# new v24 : 70
#our = ["0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v25 : 70 (tree=40 ; vote not changed)
#our = ["0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0"]
# new v26 : 68 (tree=40 ; vote choose[1])
#our = ["0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "1", "1", "0", "0", "1", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "1", "0"]
# new v27 : 74 (11 hours ; numTrees=20, impurity='gini', maxDepth=20, maxBins=32)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "1", "1", "0", "1", "0", "0", "1", "1", "1", "0", "0", "0", "0", "0", "0", "1", "0", "0", "1", "0", "0", "1", "1", "0", "1", "0", "1", "0", "1", "1", "1", "0", "1", "1", "0", "0", "1", "0", "1", "0"]
# new v28 : 71 (11 hours ; numTrees=20, impurity='gini', maxDepth=25, maxBins=32)
#our = ["0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "1", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "1", "0", "0", "1", "0", "0", "1", "1", "1", "0", "0", "1", "0", "0", "1", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "1", "0", "0", "1", "1", "0", "0", "1", "0", "1", "1", "0", "1", "0"]
# new v29 : 63 (normal 3 model ; 3 features)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "0", "0", "0", "1", "0", "0", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "0", "1", "0"]
# new v30 : 69 (pm25,pm10)
#our = ["0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "0", "1", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "1", "1", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "1", "0", "0", "1", "1", "0", "0", "1", "0", "0", "1", "1", "1", "0", "0", "1", "0", "0", "1", "1", "0", "0", "1", "0", "0", "1", "1", "0", "1", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1"]
# new v31 : 71 (pm25,pm10,temp)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "1", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "1", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "1", "1", "0", "1", "1", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "1", "0"]
# new v32 : 74 (RF training 2 times)
#our = ["0", "0", "1", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "0", "0", "0", "1", "0", "1", "1", "0", "1", "0", "0", "0", "0", "1", "1", "1", "0", "1", "1", "0", "1", "1", "0", "1", "0", "1", "1", "1", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "0", "0", "1", "0", "0", "1", "0", "1", "0", "1", "0", "0", "0", "1", "1", "0", "1", "1", "0", "1", "0"]
# new v33 : 63 (RF training 3 times)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "1", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "0", "0", "1", "1", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "1", "1", "1", "1", "0", "1", "1", "0", "0", "0", "0", "1", "0", "1", "0", "0", "1", "1", "1", "1", "1", "0", "0", "1", "1", "1", "1", "0", "1", "0", "0", "1", "1", "0", "0", "0", "0", "0", "1", "1", "0", "1", "1", "1", "0", "1", "0", "1", "0", "0", "1", "0", "1", "1", "0", "0", "0"]
# new v34 : 66 (RF training 4 times)
#our = ["0", "0", "1", "0", "1", "1", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "0", "0", "1", "0", "1", "0", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "1", "0", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "1", "0", "0", "1", "0", "0", "0", "1", "0", "0", "1", "1", "1", "1", "0", "1", "0", "0", "1", "1", "0", "0", "1", "0", "0", "1", "1", "0", "0", "1", "1", "0", "0", "1", "1", "0", "1", "1", "1", "1", "1", "0", "1", "1"]
# new v35 : 67 (RF training 5 times)
#our = ["0", "1", "1", "0", "1", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "1", "1", "0", "0", "0", "1", "0", "0", "1", "1", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "1", "1", "1", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "1", "1", "1", "0", "0", "1", "1", "0", "1", "1", "0", "0", "1", "0", "1", "1", "0", "1", "0", "0", "1", "1", "0", "0", "1", "0", "0", "1", "1", "0", "1", "1", "1", "0", "1", "0", "1", "1", "1", "1", "0", "1", "1", "0", "1", "0"]
# new v36 : 66 (RF training 6 times)
#our = ["0", "1", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "0", "1", "0", "1", "1", "0", "0", "0", "0", "0", "1", "1", "0", "1", "0", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "1", "0", "0", "1", "1", "1", "0", "1", "0", "0", "1", "0", "1", "0", "0", "1", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1", "0", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "1", "1", "0", "1", "1"]
# new v37 : 65
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "1", "0", "0", "1", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "1", "1", "0", "1", "1", "1", "1", "0", "0", "1", "0", "1", "0", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "1", "0", "0", "1", "1", "0", "0", "0", "0", "0", "1", "1", "1", "1", "0", "1", "0", "0", "1", "1", "0", "0", "1", "0", "1", "1", "1", "0", "1", "1", "1", "0", "1", "1", "1", "0", "0", "0", "0", "0", "1", "0", "1", "1"]
# new v38 : 71 (traing_data)
#our = ["0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "0", "0", "1", "1", "1", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "1", "0", "1", "1", "0", "0", "1", "0", "1", "0", "0", "1", "1", "1", "1", "1", "1", "0", "0", "0", "0", "1", "1", "1", "0", "0", "1", "1", "0", "0", "1", "0", "0", "1", "1", "1", "0", "0", "1", "0", "0", "1", "1", "0", "0", "1", "0", "0", "1", "1", "0", "1", "1", "1", "0", "1", "1", "1", "0", "1", "1", "0", "0", "1", "0", "1", "1"]
# new v39 : 56
#our = ["0", "1", "1", "0", "0", "1", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "1", "1", "1", "1", "1", "1", "0", "1", "1", "0", "1", "0", "0", "1", "1", "1", "0", "0", "1", "0", "0", "1", "1", "1", "0", "0", "1", "1", "0", "0", "0", "1", "1", "0", "1", "1", "1", "0", "1", "0", "0", "1", "0", "1", "0", "0", "0", "1", "0", "1", "0", "1", "0", "1", "1", "0", "1", "1", "0", "0", "0", "0", "1", "1", "1", "0", "0", "1", "1", "1", "1", "1"]


count = 0
for i in range(100):
  if ans[i] == our[i]:
    count += 1

print('Score', count)

In [0]:
'''
####
if predict == 1:
    print("Predict1")
    only1data = []
    onlypm10data = []
    ###################################################
    count = 0
    for index in range(-1, -25, -1):
      if ttt_data_yes[index] != "" and count != 6:
        only1data.append(ttt_data_yes[index])
        count += 1
    count = 0
    for index in range(-1, -25, -1):
      if ttt_pm10_yes[index] != "" and count != 6:
        onlypm10data.append(ttt_pm10_yes[index])
        count += 1
    ###################################################
    # data clean
    print(only1data)
    print(onlypm10data)
    print('after strip\n')
    strip_symbol(only1data)
    strip_symbol(onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after del!!')
    only1data, onlypm10data = del2big(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after fillin!!')
    only1data, onlypm10data = fillin(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after clean!!')
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6)
    
  if predict == 2:
    print("Predict2")
    only1data = []
    onlypm10data = []
    data_yes = []
    data10_yes = []
    # this day pm25 1
    temp = ttt_data[3]
    only1data.append(temp)
    # this day pm10 1
    temp10 = ttt_pm10_data[3]
    onlypm10data.append(temp10)
    
    ###################################################
    count = 0
    for index in range(-1, -25, -1):
      if ttt_data_yes[index] != "" and count != 5:
        data_yes.append(ttt_data_yes[index])
        count += 1
    count = 0
    for index in range(-1, -25, -1):
      if ttt_pm10_yes[index] != "" and count != 5:
        data10_yes.append(ttt_pm10_yes[index])
        count += 1
    ###################################################
    #data_yes = ttt_data_yes[-2:]
    #temp.extend(only1data)
    only1data.extend(data_yes)
    
    #only1data = temp

    #data10_yes = ttt_pm10_yes[-2:]
    #temp.extend(onlypm10data)
    onlypm10data.extend(data10_yes)

    #onlypm10data = temp
    # data clean
    print(only1data)
    print(onlypm10data)
    print('after clean!!')
    strip_symbol(only1data)
    strip_symbol(onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after del!!')
    only1data, onlypm10data = del2big(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after fillin!!')
    only1data, onlypm10data = fillin(only1data, onlypm10data)  
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6) 

  if predict == 3:
    print("Predict3")
    temp = []
    temp10 = []
    only1data = ttt_data[3:5]
    #print('only1data' + str(only1data))

    onlypm10data = ttt_pm10_data[3:5]
    #print('len', len(ttt_data_yes))
    
    last = ttt_data_yes[-1]
    temp.append(last)
    print(temp)
    temp.extend(only1data)
    only1data = temp

    last = ttt_pm10_yes[-1]
    temp10.append(last)
    temp10.extend(onlypm10data)
    onlypm10data = temp
    #print(type(onlypm10data))
    # data clean
    print(only1data)
    print(onlypm10data)
    print('after clean!!')
    strip_symbol(only1data)
    strip_symbol(onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after del!!')
    only1data, onlypm10data = del2big(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after fillin!!')
    only1data, onlypm10data = fillin(only1data, onlypm10data) 
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6) 

    
  if predict > 3 :
    print("Predict>3")
    ii = predict + 2
    only1data = ttt_data[ii-3:ii]

    onlypm10data = ttt_pm10_data[ii-3:ii]
    # data clean
    print(only1data)
    print(onlypm10data)
    print('after clean!!')
    strip_symbol(only1data)
    strip_symbol(onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after del!!')
    only1data, onlypm10data = del2big(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after fillin!!')
    only1data, onlypm10data = fillin(only1data, onlypm10data) 
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6)
'''

In [0]:
'''
from pyspark.mllib.regression import LabeledPoint
import time
tt_data=[]
ans = "["
total_test = 100

for i in range(total_test):
  num = i+1
  print('Round: ',num)
  if num < 10:
    file_num = '/content/drive/My Drive/colab/pm25_test/00' + str(num) + '.csv'
    #print(1)
  elif num >= 10 and num < 100:
    file_num = '/content/drive/My Drive/colab/pm25_test/0' + str(num) + '.csv'
    #print(2)
  else:
    file_num = '/content/drive/My Drive/colab/pm25_test/' + str(num) + '.csv'
    #print(3)
  print(file_num)
  # get start time
  start = time.time()

  # read file
  test_data = sc.textFile(file_num)
  #print(test_data.count())
  #test_data = test_data.map(strip_symbol)
  test_data_rdd = test_data.map(lambda line : line.split(","))
  test_data_rdd = test_data_rdd.filter(lambda x: x[2] == 'PM2.5' or x[2] == 'PM10')


  # get list
  tt_data = test_data_rdd.collect()
  # print(tt_data)
  ttt_data = tt_data[-1]  # pm25 this day
  ttt_pm10_data = tt_data[-2] #pm10 this day
  ttt_data_yes = tt_data[-3]
  ttt_pm10_yes = tt_data[-4]
  print(ttt_data)
  print(ttt_pm10_data)
  print(ttt_data_yes)
  print(ttt_pm10_yes)
  predict = 1
  for i in range(-1,-25,-1):
    if ttt_data[i] != '':
      predict = (i + 26) % 24 
      break
  print(predict)

  if predict == 1:
    print("Predict1")
    only1data = []
    onlypm10data = []
    ###################################################
    count = 0
    for index in range(-1, -25, -1):
      if ttt_data_yes[index] != "" and count != 3:
        only1data.append(ttt_data_yes[index])
        count += 1
    count = 0
    for index in range(-1, -25, -1):
      if ttt_pm10_yes[index] != "" and count != 3:
        onlypm10data.append(ttt_pm10_yes[index])
        count += 1
    ###################################################
    # data clean
    print(only1data)
    print(onlypm10data)
    print('after strip\n')
    strip_symbol(only1data)
    strip_symbol(onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after del!!')
    only1data, onlypm10data = del2big(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after fillin!!')
    only1data, onlypm10data = fillin(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after clean!!')
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6)
    
  if predict == 2:
    print("Predict2")
    only1data = []
    onlypm10data = []
    data_yes = []
    data10_yes = []
    # this day pm25 1
    temp = ttt_data[3]
    only1data.append(temp)
    # this day pm10 1
    temp10 = ttt_pm10_data[3]
    onlypm10data.append(temp10)
    
    ###################################################
    count = 0
    for index in range(-1, -25, -1):
      if ttt_data_yes[index] != "" and count != 2:
        data_yes.append(ttt_data_yes[index])
        count += 1
    count = 0
    for index in range(-1, -25, -1):
      if ttt_pm10_yes[index] != "" and count != 2:
        data10_yes.append(ttt_pm10_yes[index])
        count += 1
    ###################################################
    #data_yes = ttt_data_yes[-2:]
    #temp.extend(only1data)
    only1data.extend(data_yes)
    
    #only1data = temp

    #data10_yes = ttt_pm10_yes[-2:]
    #temp.extend(onlypm10data)
    onlypm10data.extend(data10_yes)

    #onlypm10data = temp
    # data clean
    print(only1data)
    print(onlypm10data)
    print('after clean!!')
    strip_symbol(only1data)
    strip_symbol(onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after del!!')
    only1data, onlypm10data = del2big(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after fillin!!')
    only1data, onlypm10data = fillin(only1data, onlypm10data)  
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6) 

  if predict == 3:
    print("Predict3")
    temp = []
    temp10 = []
    only1data = ttt_data[3:5]
    #print('only1data' + str(only1data))

    onlypm10data = ttt_pm10_data[3:5]
    #print('len', len(ttt_data_yes))
    last = ttt_data_yes[-1]
    temp.append(last)
    print(temp)
    temp.extend(only1data)
    only1data = temp

    last = ttt_pm10_yes[-1]
    temp10.append(last)
    temp10.extend(onlypm10data)
    onlypm10data = temp
    #print(type(onlypm10data))
    # data clean
    print(only1data)
    print(onlypm10data)
    print('after clean!!')
    strip_symbol(only1data)
    strip_symbol(onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after del!!')
    only1data, onlypm10data = del2big(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after fillin!!')
    only1data, onlypm10data = fillin(only1data, onlypm10data) 
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6) 

    
  if predict > 3 :
    print("Predict>3")
    ii = predict + 2
    only1data = ttt_data[ii-3:ii]

    onlypm10data = ttt_pm10_data[ii-3:ii]
    # data clean
    print(only1data)
    print(onlypm10data)
    print('after clean!!')
    strip_symbol(only1data)
    strip_symbol(onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after del!!')
    only1data, onlypm10data = del2big(only1data, onlypm10data)
    print(only1data)
    print(onlypm10data)
    print('after fillin!!')
    only1data, onlypm10data = fillin(only1data, onlypm10data) 
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6)

  
  only1_data = LabeledPoint(1,only1data)
  only1_data

  # Decision Tree Model
  Load_DTModel = DecisionTreeModel.load(sc, "content/drive/My Drive/colab/model/DTModel10")
  prediction_DT = Load_DTModel.predict(only1_data.features)
  print('Prediction_DT: ', int(prediction_DT))

  # Random Forest Model
  Load_RFModel = RandomForestModel.load(sc, "content/drive/My Drive/colab/model/RFModel10")
  prediction_RF = Load_RFModel.predict(only1_data.features)
  print('Prediction_RF: ', int(prediction_RF))

  # Regression Model
  Load_REModel = DecisionTreeModel.load(sc, "content/drive/My Drive/colab/model/REModel10")
  prediction_data = Load_REModel.predict(only1_data.features)
  print('Regression Model: ', prediction_data)

  # determine while >= 50
  if(prediction_data>=50):
    prediction_RE=1
  else:
    prediction_RE=0
  print('Prediction_RE: ', prediction_RE)

  choose=[]
  choose.append(int(prediction_DT))
  choose.append(int(prediction_RF))
  choose.append(prediction_RE)
  print('Choose result: ', choose)
  # vote result
  vote_result = vote(choose)
  print('Vote result: ', vote(choose))
  
  # update ans
  '''
  ans += str(vote_result)
  if num != total_test:
    ans += ","
  else:
    ans += "]"
'''
  # " "
  ans += "\""

  ans += str(vote_result)
  if num != total_test:
    ans += "\","
  else:
    ans += "\"]"

  # get end time
  end = time.time()
  print('Period: ' + str(round(end-start,2)) + 's')
  print("\n\n")
  print("Answer: ",ans)
print("\n##############################################################################\n")
print("Answer: ",ans)
'''